# Building your own algorithm container


In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM python:3.10-slim-buster

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip --no-cache-dir install numpy==1.26.4 scipy==1.12.0 scikit-learn==1.4.1.post1 pandas==1.5.3 xlsxwriter==3.2.0 open

In [2]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-spacy-inference

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  51.71kB
Step 1/12 : FROM python:3.10-slim-buster
 ---> 93b9055430ce
Step 2/12 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 641086cbe166
Step 3/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 8848a092c0e4
Step 4/12 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 0617ea12fb0e
Step 5/12 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 6008cfbaa844
Step 6/12 : RUN pip --no-cache-dir install numpy==1.26.4 scipy==1.12.0 scikit-learn==1.4.1.post1 pandas==1.5.3 xlsxwriter==3.2.0 openpyxl==3.1.2 boto3==1.34.69 flask gunicorn
 ---> Using cache
 ---> 0a4f3828febc
Step 7/12 : RUN pip install spacy==3.7.4
 ---> Using cache
 ---> 21810379edc4
Step 8/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 

### Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [3]:
### Load the spacy model
import pickle
filename = 'spacy_model.p'

nlp_ner = pickle.load(open(filename, 'rb'))


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:616: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
!tar -czvf nlp_ner.tar.gz spacy_model.p

spacy_model.p


In [8]:
import os
import boto3
import os
import boto3
import sagemaker
fObj = open("nlp_ner.tar.gz", "rb")

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
prefix = "sagemaker/spacy_deployment_custom"
key = os.path.join(prefix, "nlp_ner.tar.gz")
bucket = sess.default_bucket()

boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [9]:
bucket

'sagemaker-eu-west-2-661082688832'

In [10]:
print("key",key)

key sagemaker/spacy_deployment_custom/nlp_ner.tar.gz


In [11]:
### Create Model

from sagemaker.model import Model


uri = '661082688832.dkr.ecr.eu-west-2.amazonaws.com/sagemaker-spacy-inference'
nlp_model = Model(image_uri = uri ,model_data ='s3://sagemaker-eu-west-2-661082688832/sagemaker/spacy_deployment_custom/nlp_ner.tar.gz',role =role)

In [13]:
# from sagemaker.predictor import CSVSerializer

predictor = nlp_model.deploy(1, "ml.c4.2xlarge")

--!

### Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [15]:
import pandas as pd
shape = pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
37,setosa,4.9,3.6,1.4,0.1
38,setosa,4.4,3.0,1.3,0.2
20,setosa,5.4,3.4,1.7,0.2


In [16]:
# drop the label column in the training set
shape.drop(shape.columns[[0]], axis=1, inplace=True)
shape.sample(3)

,1,2,3,4
104,6.5,3.0,5.8,2.2
118,7.7,2.6,6.9,2.3
18,5.7,3.8,1.7,0.3


In [17]:
import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = shape.iloc[indices[:-1]]

In [18]:
test_data.values

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4],
       [4.8, 3. , 1.4, 0.3],
       [5.1, 3.8, 1.6, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [6.1, 3. , 4.6, 1.4],
       [5.8, 2.6, 4. , 1.2],
       [5. , 2.3, 3.3, 1. ],
       [5.6, 2.7, 4.2, 1.3],
       [5.7, 3. , 4.2, 1.2],
       [5.7, 2.9, 4.2, 1.3],
       [6.2, 2.9, 4.3, 1.3],
       [5.1, 2.5, 3. , 1.1],
       [5.7, 2.8, 4.1, 1.3],
       [6.7, 3.1, 5.6, 2.4],
       [6.9, 3.1, 5.1, 2.3],
       [5.8, 2.7, 5.1, 1.9],
       [6.8, 3.2, 5.9, 2.3],
       [6.7, 3.3, 5.7, 2.5],
       [6.7, 3. , 5.2, 2.3],
       [6.3, 2.5, 5. , 1.9],
       [6.5, 3. , 5.2, 2. ],
       [6.2, 3.4, 5.4, 2.3]])

In [20]:
runtime_client = boto3.client("runtime.sagemaker")


endpoint_name = 'sagemaker-spacy-inference-2024-04-07-15-03-53-270'
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body='Hello checking date is 22/10/1995'
)


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://eu-west-2.console.aws.amazon.com/cloudwatch/home?region=eu-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-spacy-inference-2024-04-07-15-03-53-270 in account 661082688832 for more information.

In [24]:
sm_client = boto3.client("sagemaker")

sm_client.delete_endpoint(EndpointName=endpoint_name)


{'ResponseMetadata': {'RequestId': 'de4b8920-5b9b-4a92-9f65-3978560d20db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'de4b8920-5b9b-4a92-9f65-3978560d20db',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 07 Apr 2024 14:07:10 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [25]:
!pwd

/home/ec2-user/SageMaker/Deelip Some Notebook/Sagemaker Model Deployment Endpoint/Sagemaker Model Deployment Bring Your Own Code/scikit_bring_your_own_2024-04-03
